# Analises Exploratoria dos Dados

## Bibliotecas

In [ ]:
# importando bibliotecas basicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Qualidade da imagem

In [ ]:
# definindo a qualidade das imagens
plt.rcParams['figure.dpi'] = 500
plt.rcParams['savefig.dpi'] = 500

## Importando os dados

In [ ]:
nomeArquivo = 'dados.xlsx'
# importa o arquivo
df_xlsx = pd.ExcelFile(nomeArquivo, engine='openpyxl')
df = pd.read_excel(df_xlsx)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Data'] = pd.to_datetime(df['Data'])
df = df.set_index(['Data'])
# remove NaN values
df = df.dropna()
# retorna df
df.head()

## Analise do intervalo entre leituras

In [ ]:
dfData = pd.DataFrame()
dfData['Data'] = df.index
dfData['Dias entre Leituras'] = dfData['Data'].diff(1)
dfData = dfData.dropna()
dfData['Dias entre Leituras num'] = dfData['Dias entre Leituras'].dt.days

In [ ]:
dfData.describe()

In [ ]:
f, ax = plt.subplots(figsize = (5, 5))
dfData['Dias entre Leituras'].astype('timedelta64[D]').plot.hist(bins=20)
plt.xlabel("Intervalo de dias entre leituras consecutivas")
plt.ylabel("Frequência")
plt.xticks(rotation=45);

In [ ]:
dfData

In [ ]:
limiteInferior = 10
limiteSuperior = 20
dfDataAnomalia = dfData.loc[(dfData['Dias entre Leituras num'] >= limiteInferior) & (dfData['Dias entre Leituras num'] <= limiteSuperior)]
numAnomalia = len(dfData)
numLeituras = len(df)
porcentagemAnomalias = numAnomalia /numLeituras
porcentagemAnomalias

In [ ]:
f, ax = plt.subplots(figsize = (5, 5))
dfDataAnomalia['Dias entre Leituras'].astype('timedelta64[D]').plot.hist(bins=20)
plt.xlabel("Intervalo de dias entre leituras consecutivas")
plt.ylabel("Frequência")
plt.xticks(rotation=45);

## Analise exploratoria das leituras

In [ ]:
df.describe()

In [ ]:
f, ax1 = plt.subplots(figsize = (6, 5))
sns.lineplot(x=df.index, y='PZA', data=df, label='PZA')
sns.lineplot(x=df.index, y='NA', data=df, label='NA')
plt.ylabel("Cota (m)");
plt.xlabel("Ano");

In [ ]:
f, ax1 = plt.subplots(figsize = (6, 5))
sns.lineplot(x=df.index, y='PZA', data=df, label='PZA')
sns.lineplot(x=df.index, y='NA', data=df, label='NA')
plt.ylabel("Elevation (m)");
plt.xlabel("Year");

## Analise de autocorrelação

In [ ]:
dfAutoCorrelacao = pd.DataFrame()
# Copia os dados
dfAutoCorrelacao['t'] = df['PZA']
# Cria as leituras do instante t-1, t-2 e t-3
dfAutoCorrelacao['t-1'] = dfAutoCorrelacao['t'].shift(1)
# remove os NaN values
dfAutoCorrelacao = dfAutoCorrelacao.dropna()
# Retorna o df de autocorrelacao
dfAutoCorrelacao.head()

In [ ]:
df.head()

In [ ]:
# analise de correlacao
matrizCorrelacao = dfAutoCorrelacao.corr()
# retorna a matrix de autocorrelacao
matrizCorrelacao

In [ ]:
# grafico de autocorrelacao dos instantes t e t-1
fig, ax1 = plt.subplots(figsize = (5, 5))
sns.scatterplot(x='t-1', y='t', data=dfAutoCorrelacao, ax=ax1)
plt.ylabel('PZA Reading at T')
plt.xlabel('PZA Reading at T-1');

## Analise da taxa de variacao

In [ ]:
dfTaxaVariacao = pd.DataFrame()
dfTaxaVariacao['x'] = df['PZA']
dfTaxaVariacao['taxaVariacao'] = dfTaxaVariacao['x'].diff(1)
dfTaxaVariacao['taxaVariacaoAbsoluta'] = abs(dfTaxaVariacao['taxaVariacao'])

In [ ]:
f, ax = plt.subplots(figsize = (5, 5))
dfTaxaVariacao['taxaVariacaoAbsoluta'].plot.hist(bins=20)
plt.xlabel("Taxa de variação absoluta entre leituras consecutivas")
plt.ylabel("Frequência")
plt.xticks(rotation=45);

In [ ]:
dfTaxaVariacao

In [ ]:
limite = 0.17
dfTaxaVariacaoAnomalia = dfTaxaVariacao.loc[dfTaxaVariacao['taxaVariacaoAbsoluta'] >= limite]
numAnomalia = len(dfTaxaVariacaoAnomalia)
numLeituras = len(df)
porcentagemAnomalias = numAnomalia /numLeituras
porcentagemAnomalias